# Data Loading and Transformation

## Learning Objectives
By the end of this notebook, you will be able to:
- Load data from various sources using pandas
- Transform data during the loading process
- Handle different data formats and encodings
- Combine data from multiple sources
- Optimize data loading for performance

## 1. Loading Data from Different Sources

In [ ]:
import pandas as pd
import numpy as np
import json
from typing import List, Dict, Any, Optional
from datetime import datetime, timedelta
import io

# Create sample data files for demonstration
def create_sample_data_files() -> None:
    """
    Create sample data files in different formats for loading examples.
    """
    
    # 1. CSV file with sales data
    sales_data = {
        'transaction_id': [f'TXN{i:04d}' for i in range(1, 101)],
        'customer_id': [f'CUST{np.random.randint(1, 21):03d}' for _ in range(100)],
        'product_name': np.random.choice(['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'], 100),
        'category': np.random.choice(['Electronics', 'Accessories'], 100),
        'quantity': np.random.randint(1, 5, 100),
        'unit_price': np.round(np.random.uniform(10, 500, 100), 2),
        'discount_percent': np.round(np.random.uniform(0, 20, 100), 1),
        'sale_date': pd.date_range('2024-01-01', periods=100, freq='D')[:100].strftime('%Y-%m-%d'),
        'sales_rep': np.random.choice(['Alice', 'Bob', 'Charlie', 'Diana'], 100)
    }
    
    df_sales = pd.DataFrame(sales_data)
    df_sales.to_csv('sales_data.csv', index=False)
    
    # 2. JSON file with customer data
    customers = []
    for i in range(1, 21):
        customer = {
            'customer_id': f'CUST{i:03d}',
            'name': f'Customer {i}',
            'email': f'customer{i}@example.com',
            'registration_date': (datetime(2023, 1, 1) + timedelta(days=i*5)).strftime('%Y-%m-%d'),
            'address': {
                'street': f'{100 + i} Main St',
                'city': np.random.choice(['New York', 'London', 'Tokyo', 'Sydney']),
                'country': 'USA'
            },
            'preferences': {
                'newsletter': np.random.choice([True, False]),
                'preferred_category': np.random.choice(['Electronics', 'Accessories'])
            }
        }
        customers.append(customer)
    
    with open('customers.json', 'w') as f:
        json.dump({'customers': customers}, f, indent=2)
    
    # 3. Tab-separated file with product data
    products_data = {
        'product_id': [f'PRD{i:03d}' for i in range(1, 11)],
        'product_name': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam', 'Tablet', 'Phone', 'Headphones', 'Speaker', 'Camera'],
        'category': ['Electronics'] * 10,
        'cost_price': [500, 15, 40, 200, 60, 300, 400, 80, 120, 250],
        'retail_price': [999, 29, 79, 399, 89, 599, 799, 149, 199, 499],
        'supplier': ['TechCorp', 'AccessoryCo', 'AccessoryCo', 'DisplayTech', 'AccessoryCo', 'TechCorp', 'MobileTech', 'AudioTech', 'AudioTech', 'PhotoTech']
    }
    
    df_products = pd.DataFrame(products_data)
    df_products.to_csv('products.tsv', sep='\t', index=False)
    
    print("Sample data files created:")
    print("✓ sales_data.csv (100 sales transactions)")
    print("✓ customers.json (20 customer records)")
    print("✓ products.tsv (10 product records)")

# Create sample files
create_sample_data_files()

In [ ]:
# Loading CSV files with different options
print("=== Loading CSV Files ===")

# Basic CSV loading
df_sales_basic: pd.DataFrame = pd.read_csv('sales_data.csv')
print(f"Basic CSV load: {df_sales_basic.shape}")
print(f"Data types:\n{df_sales_basic.dtypes}")

# CSV loading with data type specification
dtype_spec = {
    'transaction_id': 'string',
    'customer_id': 'string',
    'product_name': 'category',
    'category': 'category',
    'quantity': 'int32',
    'unit_price': 'float64',
    'discount_percent': 'float32',
    'sales_rep': 'category'
}

df_sales_typed: pd.DataFrame = pd.read_csv(
    'sales_data.csv',
    dtype=dtype_spec,
    parse_dates=['sale_date']
)

print(f"\nOptimized CSV load with types:")
print(f"Data types:\n{df_sales_typed.dtypes}")
print(f"Memory usage comparison:")
print(f"Basic: {df_sales_basic.memory_usage(deep=True).sum()} bytes")
print(f"Typed: {df_sales_typed.memory_usage(deep=True).sum()} bytes")

# Loading with column selection
selected_columns = ['transaction_id', 'customer_id', 'product_name', 'unit_price', 'sale_date']
df_sales_subset: pd.DataFrame = pd.read_csv(
    'sales_data.csv',
    usecols=selected_columns,
    parse_dates=['sale_date']
)

print(f"\nSubset loading: {df_sales_subset.shape}")
print(df_sales_subset.head())

In [ ]:
# Loading JSON files
print("=== Loading JSON Files ===")

# Load JSON file
with open('customers.json', 'r') as f:
    customers_json = json.load(f)

# Convert to DataFrame
df_customers_raw: pd.DataFrame = pd.DataFrame(customers_json['customers'])
print(f"Raw JSON DataFrame: {df_customers_raw.shape}")
print(f"Columns: {df_customers_raw.columns.tolist()}")
print(df_customers_raw.head(2))

# Normalize nested JSON data
df_customers_normalized: pd.DataFrame = pd.json_normalize(customers_json['customers'])
print(f"\nNormalized JSON DataFrame: {df_customers_normalized.shape}")
print(f"Columns: {df_customers_normalized.columns.tolist()}")
print(df_customers_normalized.head(2))

# Load JSON directly with pandas (for simple structures)
try:
    df_customers_direct: pd.DataFrame = pd.read_json('customers.json')
    print(f"\nDirect JSON load: {df_customers_direct.shape}")
except Exception as e:
    print(f"\nDirect JSON load failed: {e}")
    print("This is expected for nested JSON structures")

In [ ]:
# Loading tab-separated and other delimited files
print("=== Loading Delimited Files ===")

# Load TSV file
df_products: pd.DataFrame = pd.read_csv('products.tsv', sep='\t')
print(f"TSV file loaded: {df_products.shape}")
print(df_products.head())

# Loading with custom parameters
df_products_custom: pd.DataFrame = pd.read_csv(
    'products.tsv',
    sep='\t',
    dtype={
        'product_id': 'string',
        'product_name': 'string',
        'category': 'category',
        'supplier': 'category'
    },
    index_col='product_id'
)

print(f"\nCustom TSV load with index:")
print(df_products_custom.head())
print(f"Index: {df_products_custom.index.name}")

## 2. Data Transformation During Loading

In [ ]:
# Transform data during loading
print("=== Data Transformation During Loading ===")

# Custom date parser
def custom_date_parser(date_str: str) -> pd.Timestamp:
    """
    Custom date parser for specific date formats.
    """
    return pd.to_datetime(date_str, format='%Y-%m-%d')

# Load with transformations
df_sales_transformed: pd.DataFrame = pd.read_csv(
    'sales_data.csv',
    dtype={
        'transaction_id': 'string',
        'customer_id': 'string',
        'product_name': 'category',
        'category': 'category',
        'sales_rep': 'category'
    },
    parse_dates=['sale_date'],
    date_parser=custom_date_parser
)

# Add calculated columns during loading process
df_sales_transformed['total_amount'] = (
    df_sales_transformed['quantity'] * df_sales_transformed['unit_price']
)
df_sales_transformed['discount_amount'] = (
    df_sales_transformed['total_amount'] * df_sales_transformed['discount_percent'] / 100
)
df_sales_transformed['final_amount'] = (
    df_sales_transformed['total_amount'] - df_sales_transformed['discount_amount']
)

print(f"Transformed sales data: {df_sales_transformed.shape}")
print(df_sales_transformed[['transaction_id', 'total_amount', 'discount_amount', 'final_amount']].head())

# Custom converters for specific columns
def price_converter(price_str: str) -> float:
    """
    Convert price string to float, handling currency symbols.
    """
    if isinstance(price_str, str):
        return float(price_str.replace('$', '').replace(',', ''))
    return float(price_str)

# Example with converters (creating sample data with currency symbols)
sample_data_with_currency = """
product,price,quantity
Laptop,"$1,299.99",1
Mouse,$29.99,2
Keyboard,$79.99,1
"""

df_currency: pd.DataFrame = pd.read_csv(
    io.StringIO(sample_data_with_currency),
    converters={'price': price_converter}
)

print(f"\nData with currency conversion:")
print(df_currency)
print(f"Price column type: {df_currency['price'].dtype}")

In [ ]:
# Loading data in chunks for large files
print("=== Chunked Data Loading ===")

# Simulate processing large file in chunks
chunk_size = 25
processed_chunks = []

print(f"Processing sales data in chunks of {chunk_size} rows:")

for chunk_num, chunk in enumerate(pd.read_csv('sales_data.csv', chunksize=chunk_size), 1):
    # Process each chunk
    chunk['total_amount'] = chunk['quantity'] * chunk['unit_price']
    chunk['chunk_number'] = chunk_num
    
    # Calculate chunk statistics
    chunk_stats = {
        'chunk_number': chunk_num,
        'rows': len(chunk),
        'total_sales': chunk['total_amount'].sum(),
        'avg_transaction': chunk['total_amount'].mean()
    }
    
    processed_chunks.append(chunk_stats)
    print(f"  Chunk {chunk_num}: {len(chunk)} rows, Total: ${chunk['total_amount'].sum():.2f}")

# Combine chunk statistics
df_chunk_stats: pd.DataFrame = pd.DataFrame(processed_chunks)
print(f"\nChunk processing summary:")
print(df_chunk_stats)
print(f"Total across all chunks: ${df_chunk_stats['total_sales'].sum():.2f}")

## 3. Combining Data from Multiple Sources

In [ ]:
# Combining data from different sources
print("=== Combining Multiple Data Sources ===")

# Load all data sources
df_sales = pd.read_csv('sales_data.csv', parse_dates=['sale_date'])
df_customers = pd.json_normalize(json.load(open('customers.json'))['customers'])
df_products = pd.read_csv('products.tsv', sep='\t')

print(f"Loaded data:")
print(f"  Sales: {df_sales.shape}")
print(f"  Customers: {df_customers.shape}")
print(f"  Products: {df_products.shape}")

# Merge sales with customer data
df_sales_customers: pd.DataFrame = pd.merge(
    df_sales,
    df_customers[['customer_id', 'name', 'address.city', 'preferences.preferred_category']],
    on='customer_id',
    how='left'
)

print(f"\nSales + Customers: {df_sales_customers.shape}")
print(df_sales_customers[['transaction_id', 'customer_id', 'name', 'address.city']].head())

# Merge with product data
df_complete: pd.DataFrame = pd.merge(
    df_sales_customers,
    df_products[['product_name', 'cost_price', 'retail_price', 'supplier']],
    on='product_name',
    how='left'
)

print(f"\nComplete dataset: {df_complete.shape}")
print(f"Columns: {df_complete.columns.tolist()}")

# Calculate additional metrics with combined data
df_complete['profit_per_unit'] = df_complete['unit_price'] - df_complete['cost_price']
df_complete['total_profit'] = df_complete['profit_per_unit'] * df_complete['quantity']
df_complete['margin_percent'] = (df_complete['profit_per_unit'] / df_complete['unit_price']) * 100

print(f"\nSample of complete dataset with calculations:")
sample_cols = ['transaction_id', 'name', 'product_name', 'unit_price', 'cost_price', 'profit_per_unit', 'margin_percent']
print(df_complete[sample_cols].head())

In [ ]:
# Different types of joins
print("=== Different Join Types ===")

# Create sample datasets to demonstrate joins
df_orders = pd.DataFrame({
    'order_id': ['ORD001', 'ORD002', 'ORD003', 'ORD004'],
    'customer_id': ['CUST001', 'CUST002', 'CUST003', 'CUST999'],  # CUST999 doesn't exist in customers
    'order_amount': [150.00, 75.50, 200.00, 99.99]
})

df_customers_sample = df_customers[['customer_id', 'name', 'address.city']].head(3)

print(f"Orders data:")
print(df_orders)
print(f"\nCustomers data:")
print(df_customers_sample)

# Inner join (only matching records)
inner_join: pd.DataFrame = pd.merge(df_orders, df_customers_sample, on='customer_id', how='inner')
print(f"\nInner join: {inner_join.shape}")
print(inner_join)

# Left join (all orders, matching customers)
left_join: pd.DataFrame = pd.merge(df_orders, df_customers_sample, on='customer_id', how='left')
print(f"\nLeft join: {left_join.shape}")
print(left_join)

# Right join (all customers, matching orders)
right_join: pd.DataFrame = pd.merge(df_orders, df_customers_sample, on='customer_id', how='right')
print(f"\nRight join: {right_join.shape}")
print(right_join)

# Outer join (all records from both)
outer_join: pd.DataFrame = pd.merge(df_orders, df_customers_sample, on='customer_id', how='outer')
print(f"\nOuter join: {outer_join.shape}")
print(outer_join)

## 4. Data Loading Optimization

In [ ]:
# Data loading optimization techniques
print("=== Data Loading Optimization ===")

import time

# Create a larger sample file for performance testing
def create_large_sample_file(filename: str, num_rows: int = 10000) -> None:
    """
    Create a larger sample file for performance testing.
    """
    np.random.seed(42)
    
    large_data = {
        'id': range(1, num_rows + 1),
        'name': [f'Customer_{i}' for i in range(1, num_rows + 1)],
        'category': np.random.choice(['A', 'B', 'C', 'D'], num_rows),
        'value1': np.random.randn(num_rows),
        'value2': np.random.randn(num_rows),
        'value3': np.random.randn(num_rows),
        'date': pd.date_range('2023-01-01', periods=num_rows, freq='H').strftime('%Y-%m-%d %H:%M:%S')
    }
    
    df_large = pd.DataFrame(large_data)
    df_large.to_csv(filename, index=False)
    print(f"Created {filename} with {num_rows:,} rows")

create_large_sample_file('large_sample.csv', 10000)

# Compare different loading strategies
def time_loading_strategy(strategy_name: str, load_function) -> float:
    """
    Time a loading strategy and return execution time.
    """
    start_time = time.time()
    df = load_function()
    end_time = time.time()
    
    execution_time = end_time - start_time
    memory_usage = df.memory_usage(deep=True).sum()
    
    print(f"{strategy_name}:")
    print(f"  Time: {execution_time:.4f} seconds")
    print(f"  Memory: {memory_usage:,} bytes")
    print(f"  Shape: {df.shape}")
    
    return execution_time

# Strategy 1: Basic loading
def load_basic():
    return pd.read_csv('large_sample.csv')

# Strategy 2: Optimized data types
def load_optimized():
    return pd.read_csv(
        'large_sample.csv',
        dtype={
            'id': 'int32',
            'name': 'string',
            'category': 'category',
            'value1': 'float32',
            'value2': 'float32',
            'value3': 'float32'
        },
        parse_dates=['date']
    )

# Strategy 3: Column selection
def load_selected_columns():
    return pd.read_csv(
        'large_sample.csv',
        usecols=['id', 'category', 'value1', 'date'],
        dtype={
            'id': 'int32',
            'category': 'category',
            'value1': 'float32'
        },
        parse_dates=['date']
    )

print("\nPerformance comparison:")
time1 = time_loading_strategy("Basic loading", load_basic)
print()
time2 = time_loading_strategy("Optimized types", load_optimized)
print()
time3 = time_loading_strategy("Selected columns", load_selected_columns)

print(f"\nPerformance summary:")
print(f"Optimized types is {time1/time2:.2f}x faster than basic")
print(f"Selected columns is {time1/time3:.2f}x faster than basic")

In [ ]:
# Loading data with error handling
print("=== Error Handling During Data Loading ===")

# Create a file with some problematic data
problematic_data = """
id,name,age,salary,date
1,Alice,25,50000,2024-01-01
2,Bob,invalid_age,60000,2024-01-02
3,Charlie,30,not_a_number,2024-01-03
4,Diana,35,70000,invalid_date
5,Eve,28,55000,2024-01-05
"""

with open('problematic_data.csv', 'w') as f:
    f.write(problematic_data)

# Strategy 1: Load with error handling
def load_with_error_handling() -> pd.DataFrame:
    """
    Load data with comprehensive error handling.
    """
    try:
        # First, try to load with basic settings
        df = pd.read_csv('problematic_data.csv')
        print(f"Basic load successful: {df.shape}")
        
        # Handle data type conversions with error handling
        df['age'] = pd.to_numeric(df['age'], errors='coerce')
        df['salary'] = pd.to_numeric(df['salary'], errors='coerce')
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        
        # Report data quality issues
        print(f"\nData quality report:")
        for col in ['age', 'salary', 'date']:
            null_count = df[col].isnull().sum()
            if null_count > 0:
                print(f"  {col}: {null_count} invalid values converted to NaN")
        
        return df
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame()

df_with_errors = load_with_error_handling()
print(f"\nLoaded data with error handling:")
print(df_with_errors)
print(f"\nData types after conversion:")
print(df_with_errors.dtypes)

# Strategy 2: Skip bad lines
print(f"\n=== Alternative: Skip Bad Lines ===")
try:
    df_skip_bad = pd.read_csv(
        'problematic_data.csv',
        error_bad_lines=False,
        warn_bad_lines=True
    )
    print(f"Loaded with skip bad lines: {df_skip_bad.shape}")
except Exception as e:
    print(f"Skip bad lines approach failed: {e}")

# Clean up temporary files
import os
temp_files = ['sales_data.csv', 'customers.json', 'products.tsv', 'large_sample.csv', 'problematic_data.csv']
for file in temp_files:
    if os.path.exists(file):
        os.remove(file)

print(f"\n✓ Temporary files cleaned up")